# PICASSO'S - BUT FOR BOOKS

## Objective
The main objective of this project is to create a Recommendation System - collaborative filtering recommendation system specifically - to create an array of interesting suggestions based on the user's input.

* We could create a model that predicts your next read
* Or predict your next author

**Specific Objectives**
* To investigate the Genres with the highest readership and popularity
* To investigate the Publishers churning the highest number of books
* To investigate the Authors with a lot of publications and popularity

## Data Understanding

The data was sourced from Kaggle. 

There are 3 csv files containing the 
* Books Dataset
* Users Dataset
* Ratings Dataset

The columns in the **Books Dataset** include:
<ol>
    <li>Book-Author</li>
    <li>Book-Title</li>
    <li>ISBN</li>
    <li>Image-URL-L</li>
    <li>Image-URL-M</li>
    <li>Image-URL-S</li>
    <li>Publisher</li>
    <li>Year-Of-Publication</li>
</ol>

The columns in the **Users Dataset** are:
<ol>
    <li>Age</li>
    <li>Location</li>
    <li>User-ID</li>
</ol>

The columns in the **Ratings Dataset** are:
<ol>
    <li>Book-Rating</li>
    <li>ISBN</li>
    <li>User-ID</li>
</ol>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data

In [242]:
class DataUnderstading:
    def __init__(self, name, df='None'):
        self.df = df
        self.name = name
        
    def load_data(self, path):
        if self.df == 'None':
            self.df = pd.read_csv(path, sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)
            self.df.head()
            
        return self.df
    
    def understanding(self):
        # Shape and Columns
        print(f"DATA UNDERSTANDING OF THE {self.name} Dataset")
        print("\n\nSHAPE")
        print(f"The {self.name} dataset contains {self.df.shape[0]:,} records and {self.df.shape[1]} columns")
        print(f"The columns are { [str(col) for col in sorted(self.df.columns)] }")
        
        # Missing values 
        print("\n\nMISSING VALUES")
        print(f"\nMissing values in the {self.name} dataset:")
        print(f"{self.df.isnull().sum().sum()}")
        
        if self.df.isnull().sum().sum() > 0:
            print("\nA deeper analysis of missing values")
            for feature, val in self.df.isnull().sum().items():
                if val > 0:
                    print(f"{feature} has {val} missing values")
        
        # Missing values 
        print("\n\nDUPLICATES")
        print(f"\nDuplicate values in the {self.name} dataset:")
        print(f"{self.df[self.df.duplicated()].sum().sum()}")
        
        # Data Types
        data_types = {}
        for dt in self.df.dtypes:
            if dt not in data_types:
                data_types[dt] = 1
            else:
                data_types[dt] += 1
                
        print("\n\nDATATYPES")
        print(f"\nNumber of Features with {[dt for dt in list(data_types.keys())]} datatypes "\
          +f"are {[dt for dt in list(data_types.values())]} respectively")
        
        # Unique Values
        print("\n\nUNIQUE VALUES")
        for col in self.df.columns:
            print(f"Number of unique values in {col} are {self.df[col].nunique()}")
            
            print(f"\nSome unique values in the {col} column:")
            if self.df[col].nunique() > 11:
                print(self.df[col].unique()[:5])
            else:
                print(self.df[col].unique())
                
            print(f"\nTop 10 Sample of the Distribution of the {col} column")
            print(self.df[col].value_counts()[:10])
            
        
        # Numerical Columns
        print("\n\nCOLUMNS")
        print("\nThe Numerical Features are:")
        numerical_features = [col for col in self.df.select_dtypes(exclude='O')]
        if len(numerical_features) > 0:
            print(f"{ [col for col in numerical_features] }")
        else:
            print('None')
            
                
        # Categorical Columns
        print("\nThe Categorical Features are:")
        cat_features = [col for col in self.df.select_dtypes(include='O')]
        if len(cat_features) > 0:
            print(f"{ [col for col in cat_features] }")
        else:
            print('None')

In [173]:
# Columns
for col in sorted(ratings_df.columns):
    print(f"<li>{col}</li>")

<li>ISBN</li>
<li>rating</li>
<li>user_id</li>


#### DATASET 1 - Books

In [174]:
books = DataUnderstading(name='Books')
books_df = books.load_data(path='./data/BX-Books.csv')
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
# Describe

In [175]:
books.understanding()

DATA UNDERSTANDING OF THE Books Dataset


SHAPE
The Books dataset contains 271360 records and 8 columns
The columns are ['Book-Author', 'Book-Title', 'ISBN', 'Image-URL-L', 'Image-URL-M', 'Image-URL-S', 'Publisher', 'Year-Of-Publication']


MISSING VALUES

Missing values in the Books dataset:
7

A deeper analysis of missing values
Book-Author has 2 missing values
Publisher has 2 missing values
Image-URL-L has 3 missing values


DUPLICATES

Duplicate values in the Books dataset:
0


DATATYPES

Number of Features with [dtype('O')] datatypes are [8] respectively


UNIQUE VALUES
The number of unique values in each feature of the Books Dataset are
:
ISBN - 271360

Some unique values in the ISBN column:
['0195153448' '0002005018' '0060973129' '0374157065' '0393045218']

Top 10 Sample of the Distribution of the ISBN column
ISBN
0195153448    1
0746008481    1
0395219906    1
043916169X    1
0879235322    1
077108482X    1
0207124310    1
0439172543    1
0590408518    1
0902375512    1
Name: c

#### DATASET 2 - Users

In [176]:
users = DataUnderstading(name='Users')
users_df = users.load_data(path='./data/BX-Users.csv')
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [177]:
users.understanding()

DATA UNDERSTANDING OF THE Users Dataset


SHAPE
The Users dataset contains 278858 records and 3 columns
The columns are ['Age', 'Location', 'User-ID']


MISSING VALUES

Missing values in the Users dataset:
110762

A deeper analysis of missing values
Age has 110762 missing values


DUPLICATES

Duplicate values in the Users dataset:
0.0


DATATYPES

Number of Features with [dtype('int64'), dtype('O'), dtype('float64')] datatypes are [1, 1, 1] respectively


UNIQUE VALUES
The number of unique values in each feature of the Users Dataset are
:
User-ID - 278858

Some unique values in the User-ID column:
[1 2 3 4 5]

Top 10 Sample of the Distribution of the User-ID column
User-ID
1         1
185904    1
185910    1
185909    1
185908    1
185907    1
185906    1
185905    1
185903    1
185725    1
Name: count, dtype: int64
Location - 57339

Some unique values in the Location column:
['nyc, new york, usa' 'stockton, california, usa'
 'moscow, yukon territory, russia' 'porto, v.n.gaia, portugal

#### DATASET 3 - Ratings

In [202]:
rating = DataUnderstading(name='Rating')
ratings_df = rating.load_data(path='./data/BX-Book-Ratings.csv')
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [179]:
rating.understanding()

DATA UNDERSTANDING OF THE Rating Dataset


SHAPE
The Rating dataset contains 1149780 records and 3 columns
The columns are ['Book-Rating', 'ISBN', 'User-ID']


MISSING VALUES

Missing values in the Rating dataset:
0


DUPLICATES

Duplicate values in the Rating dataset:
0


DATATYPES

Number of Features with [dtype('int64'), dtype('O')] datatypes are [2, 1] respectively


UNIQUE VALUES
The number of unique values in each feature of the Rating Dataset are
:
User-ID - 105283

Some unique values in the User-ID column:
[276725 276726 276727 276729 276733]

Top 10 Sample of the Distribution of the User-ID column
User-ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
212898     4785
278418     4533
76352      3367
110973     3100
235105     3067
Name: count, dtype: int64
ISBN - 340556

Some unique values in the ISBN column:
['034545104X' '0155061224' '0446520802' '052165615X' '0521795028']

Top 10 Sample of the Distribution of the ISBN column
ISBN
0971880107  

In [240]:
print(f'{1149780:,}')

1,149,780


## Data Cleaning

**Validity**: 

Here we would make sure that the data fits into the correct range and format
Like we would 
- remove irrelevant columns, 
- remove white space, 
- typos in our variables.
- change data types say date to date_time.


**Accuracy** : 

Here data should represent the real world
Like a child can't be married or if we have a dataset about the cost of living in cities the total column must be equivalent to the sum of rent, transport, and food

**Completeness** : 

Here you check if all required data is present
You would check for missing values and deal with the missing values

**Consistency**: 

Make sure that the data is aligned.
Hapa ndio tutacheck duplicates,

**Uniformity**

Here data is standardized.
eg fixing messy column names,

### Validity

#### Change column names

In [181]:
# Rename columns in the books_df
books_df.rename(columns={
    'Book-Title': 'title',
    'Book-Author': 'author',
    'Year-Of-Publication': 'year',
    'Publisher': 'publisher',
    'Image-URL-L':'img_url',
}, inplace=True)

In [182]:
# Check
books_df.columns

Index(['ISBN', 'title', 'author', 'year', 'publisher', 'Image-URL-S',
       'Image-URL-M', 'img_url'],
      dtype='object')

In [226]:
# Rename columns in the users_df
users_df.rename(columns={
    'User-ID': 'user_id',
}, inplace=True)

# Check
users_df.columns

Index(['user_id', 'Location', 'Age'], dtype='object')

In [203]:
# Rename columns in the ratings_df
ratings_df.rename(columns={
    'User-ID': 'user_id',
    'Book-Rating': 'rating',
}, inplace=True)

# Check
ratings_df.columns

Index(['user_id', 'ISBN', 'rating'], dtype='object')

### Completeness

In [244]:
# Drop any records in the books and ratings df

In [ ]:
# Drop the 'age' feature in the users df

### Consistency

#### Maintain only top 200 contributing members in the ratings dataset

In [209]:
over_200_contribution = ratings_df['user_id'].value_counts() > 200
over_200_contribution = over_200_contribution[over_200_contribution == True].index
over_200_contribution

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

##### Observation
> Only 899 contributors have rated more that 200 books in the ratings dataset

In [215]:
# Create a new dataframe composed of these individuals
ratings_df = ratings_df[ratings_df['user_id'].isin(over_200_contribution)]

# Confirm changes have worked
new_ratings = DataUnderstading(name='New Ratings', df=ratings_df)
new_ratings.understanding()

DATA UNDERSTANDING OF THE New Ratings Dataset


SHAPE
The New Ratings dataset contains 526356 records and 3 columns
The columns are ['ISBN', 'rating', 'user_id']


MISSING VALUES

Missing values in the New Ratings dataset:
0


DUPLICATES

Duplicate values in the New Ratings dataset:
0


DATATYPES

Number of Features with [dtype('int64'), dtype('O')] datatypes are [2, 1] respectively


UNIQUE VALUES
Number of unique values in user_id are 899

Some unique values in the user_id column:
[277427 277478 277639 278418    254]

Top 10 Sample of the Distribution of the user_id column
user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
212898     4785
278418     4533
76352      3367
110973     3100
235105     3067
Name: count, dtype: int64
Number of unique values in ISBN are 207291

Some unique values in the ISBN column:
['002542730X' '0026217457' '003008685X' '0030615321' '0060002050']

Top 10 Sample of the Distribution of the ISBN column
ISBN
0971880107    3

## Data Preprocessing

In [253]:
ratings_df.columns, books_df.columns

(Index(['user_id', 'ISBN', 'rating'], dtype='object'),
 Index(['ISBN', 'title', 'author', 'year', 'publisher', 'Image-URL-S',
        'Image-URL-M', 'img_url'],
       dtype='object'))

#### Merging the data - Ratings and Books

In [257]:
# Confirming the common column between two dataframes
def similar_cols(df1, name1, df2, name2):
    print(f"Matching column between the {name1} and {name2} is")

    for col in df1.columns:
        if col in df2.columns:
            print(col)
            break
            
similar_cols(ratings_df, "ratings", books_df, 'books')

Matching column between the ratings and books is
ISBN


In [255]:
for col in ratings_df.columns:
    if col in users_df.columns:
        print("Matching columns between the ratings_df and users_df is")
        print(col)
        
similar_cols(ratings_df, "ratings", users_df, 'users')

Matching columns between the ratings_df and users_df is
user_id
Matching column between the ratings and users is
user_id


In [229]:
# Merging the ratings with the books dataset
ratings_with_books = ratings_df.merge(right=books_df, on='ISBN')
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...


In [243]:
# Understanding the new data
merged_rating_books = DataUnderstading('Merged Ratings and Books', df=ratings_with_books)
merged_rating_books.understanding()

DATA UNDERSTANDING OF THE Merged Ratings and Books Dataset


SHAPE
The Merged Ratings and Books dataset contains 487,671 records and 10 columns
The columns are ['ISBN', 'Image-URL-M', 'Image-URL-S', 'author', 'img_url', 'publisher', 'rating', 'title', 'user_id', 'year']


MISSING VALUES

Missing values in the Merged Ratings and Books dataset:
6

A deeper analysis of missing values
author has 1 missing values
publisher has 2 missing values
img_url has 3 missing values


DUPLICATES

Duplicate values in the Merged Ratings and Books dataset:
0


DATATYPES

Number of Features with [dtype('int64'), dtype('O')] datatypes are [2, 8] respectively


UNIQUE VALUES
Number of unique values in user_id are 899

Some unique values in the user_id column:
[277427   3363  11676  12538  13552]

Top 10 Sample of the Distribution of the user_id column
user_id
11676     11144
198711     6456
153662     5814
98391      5779
35859      5646
212898     4289
278418     3996
76352      3329
110973     2971
235105

#### How many ratings does each book have?

In [235]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()
num_rating.sort_values(by='rating', ascending=False)

,title,rating
156313,Wild Animus,363
19140,Bridget Jones's Diary,277
130224,The Lovely Bones: A Novel,270
132891,The Notebook,241
133898,The Pelican Brief,236
...,...,...
93081,Portrait of the Psychopath as a Young Woman,1
38031,"Edge of Love (Silhouette Intimate Moments, #13)",1
93078,Portrait of the Artist as a Young Man (Twentie...,1
93077,Portrait of the Artist as a Young Dog,1


##### Observations
> There are some books that received less than 50 reviews. We should only focus on books with many reviews.
> Books with 1 rating would only not be sufficient enough to judge whether it was a good book.

In [245]:
# Clean the columns
num_rating.rename(columns={
    'rating': "no_of_rating"
}, inplace=True)

#### Merging the data - `ratings_with_books` with `num_ratings` table

In [ ]:
# Merge the ratings df with the num_ratings df
ratings_with_books.merge(num_rating)